# Lakehousing of HEP data using Apache Iceberg

Jayjeet Chakraborty, University of California, Santa Cruz


## Features of Iceberg:

* Supports transactions
* Hidden Partitioning
* Schema Evolution
* Time Travel and Rollbacks
* Expressive SQL
* Views

## Adding Iceberg to Spark

The Iceberg JAR file has to be copied to the Spark installation's JAR directory.

In [1]:
!ls /opt/spark/jars | grep "iceberg"

iceberg-spark-runtime-3.3_2.12-1.1.0.jar


We need to add some configurations options for Spark to pickup Iceberg.

In [2]:
!cat /opt/spark/conf/spark-defaults.conf

spark.sql.extensions                   org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions
spark.sql.catalog.demo                 org.apache.iceberg.spark.SparkCatalog
spark.sql.catalog.demo.warehouse       warehouse
spark.sql.catalog.demo.type            hadoop
spark.sql.defaultCatalog               demo
spark.eventLog.enabled                 true
spark.eventLog.dir                     /home/iceberg/spark-events
spark.history.fs.logDirectory          /home/iceberg/spark-events
spark.sql.catalogImplementation        in-memory


In [3]:
spark

## Creating an Iceberg table from a Parquet file

We read NanoEvents data out of our Parquet file into a Spark Dataframe

In [4]:
df = spark.read.parquet("dataset")

We now create an Iceberg table out of this dataframe partitioned by the `event` field.

In [5]:
%%sql

DROP TABLE IF EXISTS hep;

23/02/22 03:03:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

For the purpose of the demo, we just use a subset of the data.

In [6]:
!ls dataset/ | wc -l

64


In [8]:
!ls queries/

query1.sql  query3.sql	query5.sql    query6-2.sql  query8.sql
query2.sql  query4.sql	query6-1.sql  query7.sql


In [9]:
df

DataFrame[run: int, luminosityBlock: bigint, event: bigint, MET: struct<pt:float,phi:float,sumet:float,significance:float,CovXX:float,CovXY:float,CovYY:float>, HLT: struct<IsoMu24_eta2p1:boolean,IsoMu24:boolean,IsoMu17_eta2p1_LooseIsoPFTau20:boolean>, PV: struct<npvs:int,x:float,y:float,z:float>, Muon: array<struct<pt:float,eta:float,phi:float,mass:float,charge:int,pfRelIso03_all:float,pfRelIso04_all:float,tightId:boolean,softId:boolean,dxy:float,dxyErr:float,dz:float,dzErr:float,jetIdx:int,genPartIdx:int>>, Electron: array<struct<pt:float,eta:float,phi:float,mass:float,charge:int,pfRelIso03_all:float,dxy:float,dxyErr:float,dz:float,dzErr:float,cutBasedId:boolean,pfId:boolean,jetIdx:int,genPartIdx:int>>, Photon: array<struct<pt:float,eta:float,phi:float,mass:float,charge:int,pfRelIso03_all:float,jetIdx:int,genPartIdx:int>>, Jet: array<struct<pt:float,eta:float,phi:float,mass:float,puId:boolean,btag:float>>, Tau: array<struct<pt:float,eta:float,phi:float,mass:float,charge:int,decayMode:

In [10]:
df.writeTo("hep").using("iceberg").create()

## Data, Metadata, WAL, and Snapshots

Data files

In [11]:
%%sql

SELECT file_path, file_size_in_bytes FROM demo.hep.all_data_files;

file_path,file_size_in_bytes
warehouse/hep/data/00000-1-6b83ef08-61a1-4bc8-ac43-14befbab349c-00001.parquet,307876
warehouse/hep/data/00001-2-152defff-4511-48d9-acbc-a4ae708a1724-00001.parquet,307876
warehouse/hep/data/00002-3-6c119db2-14b5-4789-a00b-4382f117d7e6-00001.parquet,307876
warehouse/hep/data/00003-4-6764422e-4c64-4cb1-97bf-bd43969ce749-00001.parquet,307876
warehouse/hep/data/00004-5-3b7e414b-77b1-422e-aeb1-5ba5d616325f-00001.parquet,307876
warehouse/hep/data/00005-6-0280a53c-b638-456d-a2c5-902206fe9440-00001.parquet,307876
warehouse/hep/data/00006-7-650d0b9d-ddc2-4100-954a-e13ab3862118-00001.parquet,307876
warehouse/hep/data/00007-8-079375c0-cedb-45fa-a311-78ede77e140e-00001.parquet,307876
warehouse/hep/data/00008-9-8c23170a-47b8-4fde-9249-c6a6f51a3895-00001.parquet,307876
warehouse/hep/data/00009-10-d0e9d398-43af-476f-999c-e065997e2b8b-00001.parquet,307876


Metadata files

In [12]:
%%sql

SELECT * FROM demo.hep.manifests;

content,path,length,partition_spec_id,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_delete_files_count,existing_delete_files_count,deleted_delete_files_count,partition_summaries
0,warehouse/hep/metadata/ad92e2fb-d43b-47a6-bf26-a4bfe5bd7be7-m0.avro,14048,0,6027879554434080617,32,0,0,0,0,0,[]


Write-Ahead Log files that enables transactions

In [13]:
%%sql

SELECT * from demo.hep.metadata_log_entries;

timestamp,file,latest_snapshot_id,latest_schema_id,latest_sequence_number
2023-02-22 03:03:44.397000,warehouse/hep/metadata/v1.metadata.json,6027879554434080617,0,0


Snapshot files

In [14]:
%%sql

SELECT snapshot_id, manifest_list FROM demo.hep.snapshots

snapshot_id,manifest_list
6027879554434080617,warehouse/hep/metadata/snap-6027879554434080617-1-ad92e2fb-d43b-47a6-bf26-a4bfe5bd7be7.avro


## Running ADL benchmark queries on Iceberg

https://github.com/iris-hep/adl-benchmarks-index/

https://arxiv.org/pdf/2104.12615.pdf

In [19]:
import time

for query_id in ["1", "2", "3", "4", "5", "6-1", "6-2", "7", "8"]:
    with open(f"queries/query{query_id}.sql") as f:
        query = f.read()
        query = query.replace("{table}", "hep")
        s = time.time()
        resp = spark.sql(query).collect()
        print(resp)
        e = time.time()
        print(f"Query {query_id}: ", e-s)

[Row(x=10, y=22144), Row(x=30, y=27328), Row(x=50, y=12544), Row(x=70, y=1600), Row(x=90, y=64), Row(x=110, y=256), Row(x=130, y=64)]
Query 1:  0.1007835865020752
[Row(x=Decimal('15.225'), y=6144), Row(x=Decimal('15.675'), y=4864), Row(x=Decimal('16.125'), y=3520), Row(x=Decimal('16.575'), y=4224), Row(x=Decimal('17.025'), y=3776), Row(x=Decimal('17.475'), y=3328), Row(x=Decimal('17.925'), y=2496), Row(x=Decimal('18.375'), y=3776), Row(x=Decimal('18.825'), y=2816), Row(x=Decimal('19.275'), y=2560), Row(x=Decimal('19.725'), y=1920), Row(x=Decimal('20.175'), y=1600), Row(x=Decimal('20.625'), y=1728), Row(x=Decimal('21.075'), y=2240), Row(x=Decimal('21.525'), y=1792), Row(x=Decimal('21.975'), y=1216), Row(x=Decimal('22.425'), y=1664), Row(x=Decimal('22.875'), y=1152), Row(x=Decimal('23.325'), y=1728), Row(x=Decimal('23.775'), y=1216), Row(x=Decimal('24.225'), y=1024), Row(x=Decimal('24.675'), y=1280), Row(x=Decimal('25.125'), y=1472), Row(x=Decimal('25.575'), y=1088), Row(x=Decimal('26.02